In [1]:
variant <- 5
set.seed(variant)
k <- sample(c(4:9), 1)
pp1 <- runif(4)
pp2 <- runif(3)
pp3 <- runif(2)
p1 <- pp1 / sum(pp1)
p2 <- c(c(0), pp2 / sum(pp2))
p3 <- c(c(0, 0), pp3 / sum(pp3))
p4 <- c(0, 0, 0, 1)
P <- data.frame()
P <- rbind(P, p1)
P <- rbind(P, p2)
P <- rbind(P, p3)
P <- rbind(P, p4)
rownames(P) <- c("p1", "p2", "p3", "p4")
colnames(P) <- c("", "", "", "")
View(P)
print(paste("k =", as.character(k)))

,,,,
,<dbl>,<dbl>,<dbl>,<dbl>
p1,0.3441331,0.4604769,0.1428322,0.05255779
p2,0.0000000,0.3441697,0.2591911,0.39663917
p3,0.0000000,0.0000000,0.8964781,0.10352190
p4,0.0000000,0.0000000,0.0000000,1.00000000


[1] "k = 5"


In [2]:
# Получаем значение, соответствующее отрезку.
# v - случайная величина. Если нужно сгенерировать распределение по `p`, нужно
# подать runif(1)
# X - величины, соответствующие отрезкам.
# За отрезки отвечает параметр p, хранящий последовательность длин.
get_value_in_range <- function(v, X, p) {
    border <- 0
    for (i in seq_along(p)) {
        border <- border + p[i]
        if (v < border) {
            return(X[i])
        }
    }
}

In [3]:
make_step <- function(current_state) {
    possible_states <- P[current_state, ]

    return(get_value_in_range(runif(1), 1:4, possible_states))
}

[1] 1

In [4]:
walk <- function(starting_state, times) {
    if (times < 0) {
        return(starting_state)
    }

    next_state <- make_step(starting_state)

    return(walk(next_state, times - 1))
}

In [5]:
N <- 1000
test_walk <- function(first_state, k) {
    results <- c()

    for (i in 1:N) {
        results <- append(results, walk(first_state, k))
    }

    return(results)
}

In [6]:
first_state <- 1

In [7]:
S1 <- test_walk(first_state, k - 2)

In [8]:
S2 <- test_walk(first_state, k - 1)

In [9]:
S3 <- test_walk(first_state, k)

In [23]:
get_probability <- function(States, state) {
    number_of_states <- length(States[States == state])
    number_of_all_states <- length(States)

    return(number_of_states / number_of_all_states)
}

In [24]:
get_probabilities_to_stay <- function(States) {
    return(
        lapply(seq_along(P), function(state) get_probability(States, state))
    )
}

In [12]:
get_probabilities_to_stay(S1)

[[1]]
[1] 0.016

[[2]]
[1] 0.074

[[3]]
[1] 0.35

[[4]]
[1] 0.56

In [13]:
get_probabilities_to_stay(S2)

[[1]]
[1] 0.003

[[2]]
[1] 0.033

[[3]]
[1] 0.372

[[4]]
[1] 0.592

In [14]:
get_probabilities_to_stay(S3)

[[1]]
[1] 0.002

[[2]]
[1] 0.017

[[3]]
[1] 0.352

[[4]]
[1] 0.629

# # Задание 2

In [15]:
variant <- 5
set.seed(variant)
k <- sample(c(10:25), 1)
t1 <- sample(c(14:20), 1)
t2 <- sample(c(2:5), 1)
View(data.frame(k, t1, t2))

k,t1,t2
<int>,<int>,<int>
11,16,2


In [16]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

set.seed(42)

env <- simmer("SuperDuperSim")
env

Loading required package: simmer

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
"there is no package called 'simmer'"
Installing package into 'C:/Users/Александр/scoop/persist/r/site-library'
(as 'lib' is unspecified)

also installing the dependencies 'Rcpp', 'BH'




package 'Rcpp' successfully unpacked and MD5 sums checked
package 'BH' successfully unpacked and MD5 sums checked
package 'simmer' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Александр\AppData\Local\Temp\RtmpqMZumL\downloaded_packages


simmer environment: SuperDuperSim | now: 0 | next: 
{ Monitor: in memory }

In [17]:
programmers <- trajectory("programmers' path") %>%
    ## add an intake activity
    seize("server", 1) %>%
    timeout(function() rnorm(1, 5)) %>%
    release("server", 1)

In [18]:
env %>%
    add_resource("server", 1) %>%
    add_generator("programmers", programmers, function() {
        return(10)
    })
# , function() rnorm(1, 0.1, 2))

simmer environment: SuperDuperSim | now: 0 | next: 0
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 0(1) | queue status: 0(Inf) }
{ Source: programmers | monitored: 1 | n_generated: 0 }

In [19]:
env %>%
    run(50) %>%
    now()
env %>% peek(3)

[1] 50

[1] 50 50

In [20]:
env %>% get_n_generated("programmers")

[1] 5

In [21]:
env %>% get_queue_count("server")

[1] 0

In [22]:
env %>% get_mon_resources()

resource,time,server,queue,capacity,queue_size,system,limit,replication
<chr>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>
server,10.00000,1,0,1,Inf,1,Inf,1
server,16.37096,0,0,1,Inf,0,Inf,1
server,20.00000,1,0,1,Inf,1,Inf,1
server,24.43530,0,0,1,Inf,0,Inf,1
server,30.00000,1,0,1,Inf,1,Inf,1
server,35.36313,0,0,1,Inf,0,Inf,1
server,40.00000,1,0,1,Inf,1,Inf,1
server,45.63286,0,0,1,Inf,0,Inf,1
